In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_885.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_698.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1176.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1215.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_66.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_386.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_1026.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_330.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_72.wav
/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test/audio_858.wav
/kaggle/input/shl-intern-hiring-ass

In [2]:
import pandas as pd
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt

**path for input files**

In [3]:
train_csv_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv"
test_csv_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv"
train_audio_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train"
test_audio_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test"
submission_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/sample_submission.csv"

**Loading CSV'S**

In [4]:
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
print(f"Train samples: {len(train_df)}, Test samples: {len(test_df)}")
train_df.head()


Train samples: 444, Test samples: 195


,filename,label
0,audio_1261.wav,1.0
1,audio_942.wav,1.5
2,audio_1110.wav,1.5
3,audio_1024.wav,1.5
4,audio_538.wav,2.0


In [5]:
print(train_df.columns)


Index(['filename', 'label'], dtype='object')


**Featre Extraction**

In [6]:
def extract_mfcc(file_path, n_mfcc=20):
    y, sr = librosa.load(file_path, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

X = []
y = []

for _, row in train_df.iterrows():
    file_path = os.path.join(train_audio_path, row['filename'])
    mfcc = extract_mfcc(file_path)
    X.append(mfcc)
    y.append(row['label'])

X = np.array(X)
y = np.array(y)

print("Shape of features:", X.shape)

Shape of features: (444, 20)


**Train LightGBM Regressor**


In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


model = LGBMRegressor()
model.fit(X_train, y_train)


y_pred = model.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
print(f"Validation MSE: {mse:.4f}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003355 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2386
[LightGBM] [Info] Number of data points in the train set: 355, number of used features: 20
[LightGBM] [Info] Start training from score 3.635211
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

**Prediction on Test Set**

In [8]:
X_test = []
for fname in test_df['filename']:
    file_path = os.path.join(test_audio_path, fname)
    mfcc = extract_mfcc(file_path)
    X_test.append(mfcc)

X_test = np.array(X_test)
test_preds = model.predict(X_test)
submission = pd.DataFrame({
    'filename': test_df['filename'],
    'label': test_preds
})
submission.to_csv("/kaggle/working/submission.csv", index=False)
print("saved to output/submission.csv")
display(submission)

saved to output/submission.csv


,filename,label
0,audio_706.wav,2.994899
1,audio_800.wav,2.577510
2,audio_68.wav,3.492239
3,audio_1267.wav,3.489266
4,audio_683.wav,2.718299
...,...,...
190,audio_135.wav,3.636420
191,audio_512.wav,4.677834
192,audio_529.wav,3.244057
193,audio_762.wav,3.822683
